<a href="https://colab.research.google.com/github/AntonioHenriqueAC/tcc/blob/master/google_colab/Barcode_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Baixando as bibliotecas necessárias

In [0]:
!pip3 install tensorflow-gpu==1.13.1

In [0]:
!pip3 install imageai --upgrade

# 2. Baixando a CNN pré-treinada

In [0]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

# 3. Conectando o drive ao colab


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# 4. Carregando os arquivos do colab

In [0]:
!unzip "/content/drive/My Drive/Colab/barcode.zip" 

# 5. Treinando o modelo

In [0]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="barcode")
trainer.setTrainConfig(object_names_array=["barcode"], batch_size=4, num_experiments=10, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

# 6. Avaliando os modelos gerados

In [0]:
metrics = trainer.evaluateModel(model_path="barcode/models", json_path="barcode/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

Starting Model evaluation....


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Model File:  toinho/models/detection_model-ex-001--loss-0024.190.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.0768
mAP: 0.0768
Model File:  toinho/models/detection_model-ex-002--loss-0005.224.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.2622
mAP: 0.2622
Model File:  toinho/models/detection_model-ex-003--loss-0003.973.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.9334
mAP: 0.9334
Model File:  toinho/models/detection_model-ex-004--loss-0003.257.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.5655
mAP: 0.5655
Model File:  toinho/models/detection_model-ex-005--loss-0002.975.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.8350
mAP: 0.8350
Model File:  toinho/models/detection_model-ex-006--loss-0002.623.h5 

Using IoU :  0.5
Usi

# 7. Testando o modelo treinado

In [0]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("barcode/models/detection_model-ex-007--loss-0002.457.h5")
detector.setJsonPath("barcode/json/detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image="img1.jpg", output_image_path="barcode-detected.jpg")
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])

barcode  :  52.46580243110657  :  [1012, 2608, 1197, 2780]
barcode  :  51.347869634628296  :  [1759, 2599, 1938, 2774]
barcode  :  53.176701068878174  :  [1009, 2986, 1180, 3159]
barcode  :  51.97243094444275  :  [2786, 3198, 2984, 3378]
barcode  :  58.54182839393616  :  [1537, 1944, 1693, 2063]
barcode  :  59.1326117515564  :  [728, 2293, 875, 2413]
barcode  :  51.98236703872681  :  [1452, 2310, 1586, 2413]
barcode  :  59.56333875656128  :  [2151, 2254, 2269, 2392]
barcode  :  52.919572591781616  :  [2590, 2260, 2711, 2369]
barcode  :  67.47114062309265  :  [265, 2347, 411, 2455]
barcode  :  61.78871989250183  :  [1402, 2598, 1534, 2730]
barcode  :  52.16687321662903  :  [3248, 2591, 3384, 2712]
barcode  :  63.53941559791565  :  [294, 2646, 412, 2776]
barcode  :  53.743976354599  :  [3171, 2873, 3304, 3009]
barcode  :  59.57975387573242  :  [282, 2989, 423, 3126]
barcode  :  77.24733352661133  :  [1017, 2948, 1155, 3063]
barcode  :  61.43640875816345  :  [1771, 2954, 1888, 3096]
barco

In [0]:
len(detections)

49